In [1]:
from rasa.cli.utils import get_validated_path
from rasa.model import get_model, get_model_subdirectories
from rasa.nlu.model import Interpreter

def load_interpreter(model_path):
  """
  This loads the Rasa NLU interpreter. It is able to apply all NLU
  pipeline steps to a text that you provide it.
  """
  model = get_validated_path(model_path, "model")
  model_path = get_model(model)
  _, nlu_model = get_model_subdirectories(model_path)
  return Interpreter.load(nlu_model)

nlu_interpreter = load_interpreter("20220626-085810.tar.gz")

2022-06-26 09:32:38.067253: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-26 09:32:38.583338: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
/opt/anaconda3/envs/rasa/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/cond_grad/gradients/cond/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/cond_grad/gradients/cond/GatherV2_grad/Reshape:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradients/cond_grad/gradients/cond/GatherV2_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor o

In [2]:
nlu_interpreter.pipeline

In [4]:
import rasa.shared.nlu.training_data.loading as nlu_loading
train_data = nlu_loading.load_data("nlu.yml")

In [5]:
import pandas as pd

def add_predictions(dataf, nlu):
    """This function will add prediction columns based on `nlu`"""
    pred_blob = [nlu.parse(t)['intent'] for t in dataf['text']]
    return (dataf
            [['text', 'intent']]
            .assign(pred_intent=[p['name'] for p in pred_blob])
            .assign(pred_confidence=[p['confidence'] for p in pred_blob]))

# First make a list of dictionaries that contain the utterances.
data_list = [m.as_dict() for m in train_data.intent_examples]
# Turn this list into a dataframe and add predictions using the nlu-interpreter
df_intents = pd.DataFrame(data_list).pipe(add_predictions, nlu=nlu_interpreter)


In [6]:
df_summary = (df_intents
 .groupby('pred_intent')
 .agg(n=('pred_confidence', 'size'),
      mean_conf=('pred_confidence', 'mean')))

In [7]:
df_summary

,n,mean_conf
pred_intent,,
affirm,6,0.974539
bot_challenge,9,0.998295
complaints,44,0.999965
connections,27,0.999850
course_details,19,0.998550
course_price,11,0.997055
deny,7,0.996808
goodbye,10,0.998888
greet,14,0.971911


In [8]:
from sklearn.metrics import classification_report
report = classification_report(y_true=df_intents['intent'], y_pred=df_intents['pred_intent'])
print(report)

                          precision    recall  f1-score   support

                  affirm       1.00      1.00      1.00         6
           bot_challenge       1.00      1.00      1.00         9
              complaints       1.00      1.00      1.00        44
             connections       1.00      1.00      1.00        27
          course_details       1.00      1.00      1.00        19
            course_price       1.00      1.00      1.00        11
                    deny       1.00      1.00      1.00         7
                 goodbye       1.00      0.91      0.95        11
                   greet       0.93      1.00      0.96        13
                 ineuron       1.00      1.00      1.00         6
internship_opportunities       1.00      1.00      1.00        15
            join_courses       1.00      1.00      1.00        30
              mood_great       1.00      1.00      1.00        14
            mood_unhappy       1.00      1.00      1.00        14
         

In [10]:
!pip install altair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 KB 2.9 MB/s eta 0:00:00m eta 0:00:010:01:01
You should consider upgrading via the '/opt/anaconda3/envs/rasa/bin/python -m pip install --upgrade pip' command.


In [11]:
import altair as alt
bars = alt.Chart(df_intents).mark_bar().encode(
    x='pred_confidence:Q',
    y="pred_intent:O"
)

(bars).properties(height=100)

alt.Chart(...)

In [13]:
df_conf_mat = (df_intents
  .groupby(["intent", "pred_intent"])
  .agg(n_pred=("pred_confidence", "size"))
  .reset_index())

df_plot = df_conf_mat.merge(df_summary.reset_index()).assign(p=lambda d: d['n_pred']/d['n'])

alt.Chart(df_plot).mark_rect().encode(
  x='intent:O',
  y='pred_intent:O',
  color='p:Q'
)

alt.Chart(...)